In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
import os
import zipfile
import pandas as pd

In [ ]:
# Define paths
ZIP_FILE = "/content/drive/MyDrive/AI_Files/AIA-image-classification-6.zip"  # Update with the actual ZIP file path
DATA_FOLDER = "/content/drive/MyDrive/AI_Files/dataset"   # Folder where data will be extracted


In [ ]:

# Ensure Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

# Extract if not already extracted
if not os.path.exists(DATA_FOLDER):
    with zipfile.ZipFile(ZIP_FILE, "r") as zip_ref:
        zip_ref.extractall(DATA_FOLDER)

print(" Extraction complete!")

In [ ]:
for root, dirs, files in os.walk(DATA_FOLDER):
    for file in files:
        if file.endswith(".csv"):
            print(f"Found CSV file: {os.path.join(root, file)}")

In [ ]:
import os

file_path = "/content/drive/MyDrive/AI_Files/dataset/AIA image classification/upload/images.csv"

# Check if the file exists
if os.path.exists(file_path):
    print("✅ File found!")
else:
    print("❌ File not found! Check the path.")


In [ ]:
# Assuming the CSV file is inside the extracted folder
CSV_FILE = os.path.join(DATA_FOLDER, "/content/drive/MyDrive/AI_Files/dataset/AIA image classification/upload/images.csv")  # Update the actual filename

# Load the CSV file
df = pd.read_csv(CSV_FILE)

# Verify the first few rows
print(df.head())


In [ ]:
IMAGE_DIR = "/content/drive/MyDrive/AI_Files/dataset/AIA image classification/upload/images/images"  # Update this path

# Update image paths inside DataFrame
df["image_path"] = df["image_name"].apply(lambda x: os.path.join(IMAGE_DIR, x))

# Verify
print(df.head())


In [ ]:
# Remove rows where image does not exist
df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

print(f"✅ Total images after cleaning: {len(df)}")


In [ ]:
# label encoder
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])

# Split into train & validation
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label_encoded"], random_state=42)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")


In [ ]:
# data preprocessing and traing dataset
IMG_SIZE = (224, 224)  # Image size for model
BATCH_SIZE = 32

# Function to load and preprocess images
def load_and_preprocess_image(img_path, label):
    img = load_img(img_path, target_size=IMG_SIZE)  # Resize image
    img = img_to_array(img) / 255.0  # Normalize
    return img, label

# Convert DataFrame to tf.data.Dataset
def dataframe_to_tf_dataset(df, batch_size):
    file_paths = df["image_path"].values
    labels = df["label_encoded"].values

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: tf.py_function(func=load_and_preprocess_image, inp=[x, y], Tout=(tf.float32, tf.int32)))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create train & validation datasets
train_ds = dataframe_to_tf_dataset(train_df, BATCH_SIZE)
val_ds = dataframe_to_tf_dataset(val_df, BATCH_SIZE)

print("✅ Dataset ready for training!")


In [ ]:
IMG_SIZE = (224, 224)  # Image size for model
BATCH_SIZE = 32

def load_and_preprocess_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decodes JPEG image
    img = tf.image.resize(img, IMG_SIZE) / 255.0  # Resize & normalize
    return img, label

def dataframe_to_tf_dataset(df, batch_size):
    file_paths = df["image_path"].values
    labels = df["label_encoded"].values.astype(np.int32)

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    def _parse_function(file_path, label):
        img, label = tf.py_function(load_and_preprocess_image, [file_path, label], [tf.float32, tf.int32])
        img.set_shape((*IMG_SIZE, 3))  # Explicitly setting image shape
        label.set_shape([])  # Scalar shape for label
        return img, label

    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Create train & validation datasets
train_ds = dataframe_to_tf_dataset(train_df, BATCH_SIZE)
val_ds = dataframe_to_tf_dataset(val_df, BATCH_SIZE)

print(" Dataset ready for training!")

In [ ]:
#Model Development and Compilation
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load pre-trained VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(6, activation="softmax")(x)  # 6 classes

# Define model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print("✅ Model compiled successfully!")


In [ ]:
# Import necessary callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


In [ ]:
# Train the model
EPOCHS = 1

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Evaluate the model
results = model.evaluate(val_ds)
print(f"Validation Loss: {results[0]:.4f}")
print(f"Validation Accuracy: {results[1]*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.legend()
plt.show()

In [ ]:
save_path = "/content/drive/MyDrive/AI_Files/dataset/image_classification_model.keras"
model.save(save_path)
print("Fine-tuned model saved successfully in Keras format!")


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok
!ngrok config add-authtoken 2tGY0JW2AyjO8NpMuMLl3Scodih_4VZ6U3ifdsWxXzf4US6CW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import numpy as np
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Initialize Flask
app = Flask(__name__)

# Define the upload folder
UPLOAD_FOLDER = "/content/drive/MyDrive/AI_Files/dataset/UPLOAD_FOLDER"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# Load trained model
model_path = "/content/drive/MyDrive/AI_Files/dataset/image_classification_model.keras"
model = load_model(model_path)

# Function to preprocess and predict
def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence_score = np.max(predictions) * 100

    return predicted_class, confidence_score

# Home Route - Serves HTML
@app.route("/")
def home():
    return"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flask Image Classification</title>
    <style>
        * {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        body {
            background: #f4f4f4;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            text-align: center;
        }
        .container {
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.2);
            width: 90%;
            max-width: 400px;
            position: relative;
        }
        h1 {
            margin-bottom: 15px;
        }
        .upload-box {
            border: 2px dashed #007BFF;
            padding: 20px;
            cursor: pointer;
            display: block;
            margin: 10px 0;
        }
        input[type="file"] {
            display: none;
        }
        button {
            background: #007BFF;
            color: white;
            border: none;
            padding: 10px 15px;
            cursor: pointer;
            margin-top: 10px;
            width: 100%;
            border-radius: 5px;
            display: none;
        }
        .result {
            margin-top: 15px;
            font-weight: bold;
        }
        .uploaded-image {
            width: 100%;
            max-height: 200px;
            margin-top: 10px;
            border-radius: 5px;
            opacity: 0;
            transition: opacity 2s ease-in-out;
        }
        .upload-success {
            font-size: 16px;
            color: green;
            font-weight: bold;
            display: none;
            margin-top: 10px;
        }
    </style>
</head>
<body>

    <div class="container">
        <h1>Image Classification 6 Labels</h1>

        <form id="uploadForm" enctype="multipart/form-data">
            <label class="upload-box" id="uploadBox">
                Click to Upload Image
                <input type="file" id="fileInput" name="file" accept="image/*" required>
            </label>
        </form>

        <div class="upload-success" id="uploadSuccess">Image Uploaded Successfully!</div>

        <img id="uploadedImage" class="uploaded-image" alt="Uploaded Image">

        <button type="submit" id="uploadBtn">Predict</button>

        <div class="result" id="result"></div>
    </div>

    <script>
        document.getElementById("fileInput").onchange = function(event) {
            let file = event.target.files[0];
            if (!file) return;

            // Show upload success message
            let uploadSuccess = document.getElementById("uploadSuccess");
            uploadSuccess.style.display = "block";

            // Display the uploaded image
            let uploadedImage = document.getElementById("uploadedImage");
            uploadedImage.src = URL.createObjectURL(file);

            // Start the fade-in effect for the image
            setTimeout(() => {
                uploadedImage.style.opacity = "1";
            }, 1000);

            // Show Predict button after image is fully visible
            setTimeout(() => {
                document.getElementById("uploadBtn").style.display = "block";
            }, 3000);
        };

        document.getElementById("uploadBtn").onclick = async function(event) {
            event.preventDefault();

            let fileInput = document.getElementById("fileInput");
            let file = fileInput.files[0];

            if (!file) {
                alert("Please upload an image first.");
                return;
            }

            let formData = new FormData();
            formData.append("file", file);

            let response = await fetch("/predict", {
                method: "POST",
                body: formData
            });

            let result = await response.json();
            document.getElementById("result").innerText = "Prediction: " + result.prediction + " - Confidence: " + result.confidence;
        };
    </script>

</body>
</html>


    """

# Prediction Route
@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No file selected"}), 400

    try:
        # Save the uploaded file
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config["UPLOAD_FOLDER"], filename)
        file.save(file_path)

        # Make a prediction
        predicted_class, confidence_score = model_predict(file_path, model)

        return jsonify({
            "prediction": f"Class {predicted_class}",
            "confidence": f"{confidence_score:.2f}%"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start Flask with Ngrok
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)
    app.run(port=5000)


ValueError: File not found: filepath=/content/drive/MyDrive/AI_Files/dataset/image_classification_model.keras. Please ensure the file is an accessible `.keras` zip file.